In [1]:
import psycopg2
from psycopg2 import sql
import config
import utility_funcs as tsp
import pandas as pd


In [2]:
db_credentials = config.db_credentials

In [3]:
def get_unique_ship_type_ids(db_credentials):
    """
    Fetches all unique ship_type_id values from the daily_ship_destructions table.

    Parameters:
    db_credentials (dict): Database connection credentials including dbname, user, password, host, and port.

    Returns:
    list: A list of unique ship_type_id values.
    """
    try:
        # Connect to the database
        conn = psycopg2.connect(**db_credentials)
        cur = conn.cursor()
        
        # Execute the query to get unique ship_type_ids
        cur.execute(sql.SQL("SELECT DISTINCT ship_type_id FROM daily_ship_destructions"))
        
        # Fetch all unique ship_type_ids
        unique_ship_type_ids = cur.fetchall()
        
        # Convert the results into a simple list
        unique_ship_type_ids_list = [row[0] for row in unique_ship_type_ids]
        
        return unique_ship_type_ids_list

    except Exception as e:
        print(f"Database connection failed due to {e}")
        return []

    finally:
        # Clean up the database connections
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()

In [4]:
unique_ship_type_ids = get_unique_ship_type_ids(db_credentials)
print(len(unique_ship_type_ids))

Database connection failed due to relation "daily_ship_destructions" does not exist
LINE 1: SELECT DISTINCT ship_type_id FROM daily_ship_destructions
                                          ^

0


In [34]:
def get_average_prices_for_ship_types(db_credentials, type_ids=None):
    """
    Fetches average prices for given ship type IDs from the daily market prices table and returns them as a DataFrame.

    Parameters:
    db_credentials (dict): Database connection credentials including dbname, user, password, host, and port.
    type_ids (list): Optional. A list of ship type IDs to filter the query. If None, fetches from get_unique_ship_type_ids.

    Returns:
    DataFrame: A pandas DataFrame with columns ['type_id', 'average'] containing the fetched data.
    """
    if type_ids is None:
        type_ids = get_unique_ship_type_ids(db_credentials)
    
    try:
        # Connect to the database
        conn = psycopg2.connect(**db_credentials)
        cur = conn.cursor()
        
        # Prepare and execute the query
        # query = sql.SQL("SELECT date, region_id, type_id, volume, order_count, average FROM market_history WHERE type_id = ANY(%s)")
        query = sql.SQL("""
            SELECT date, region_id, type_id, volume, order_count, average
            FROM market_history
            WHERE type_id = ANY(%s) AND region_id = 10000001
        """)
        cur.execute(query, (type_ids,))
        
        # Fetch the results
        results = cur.fetchall()
        
        # Convert the results to a DataFrame
        df = pd.DataFrame(results, columns=['date', 'region_id', 'type_id', 'volume', 'order_count', 'average'])
        
        return df

    except Exception as e:
        print(f"Failed to fetch data due to {e}")
        # Return an empty DataFrame with the appropriate columns if an error occurs
        return pd.DataFrame(columns=['date', 'region_id', 'type_id', 'volume', 'order_count', 'average'])

    finally:
        # Clean up the database connections
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()


In [35]:
average_prices = get_average_prices_for_ship_types(db_credentials)  # or pass unique_ship_type_ids explicitly


In [36]:
average_prices

,date,region_id,type_id,volume,order_count,average
0,2020-12-31,10000001,583,1,1,100000
1,2020-12-31,10000001,586,1,1,100000
2,2020-12-31,10000001,587,1,1,390000
3,2020-12-31,10000001,591,1,1,400000
4,2020-12-31,10000001,593,18,1,200000
...,...,...,...,...,...,...
20262,2020-07-29,10000001,12735,1,1,50000000
20263,2020-07-29,10000001,12753,1,1,300900000
20264,2020-07-29,10000001,23757,2,2,1099500000
20265,2020-07-29,10000001,23911,1,1,1000000000


In [30]:
def get_faction_warfare_stats_by_faction(db_credentials):
    """
    Queries the faction_warfare_stats_history table and returns a dictionary of DataFrames for each unique faction_id.

    Parameters:
    db_credentials (dict): Database connection credentials including dbname, user, password, host, and port.

    Returns:
    dict: A dictionary where each key is a faction_id and each value is a DataFrame containing all entries for that faction_id.
    """
    faction_dataframes = {}

    try:
        # Connect to the database
        conn = psycopg2.connect(**db_credentials)
        cur = conn.cursor()

        # Get unique faction_ids
        cur.execute("SELECT DISTINCT faction_id FROM faction_warfare_stats_history")
        faction_ids = cur.fetchall()

        for faction_id in faction_ids:
            # Query all entries for the current faction_id
            query = """
                SELECT * FROM faction_warfare_stats_history WHERE faction_id = %s
            """
            cur.execute(query, (faction_id,))
            rows = cur.fetchall()
            
            # Assuming you know the column names and order, if not you should fetch them as well
            column_names = [desc[0] for desc in cur.description]
            
            # Create a DataFrame for the current faction_id and add it to the dictionary
            df = pd.DataFrame(rows, columns=column_names)
            faction_dataframes[faction_id[0]] = df

    except Exception as e:
        print(f"Failed to fetch data due to {e}")
        return {}

    finally:
        # Clean up the database connections
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()

    return faction_dataframes, faction_ids

In [31]:
faction_dfs, faction_ids = get_faction_warfare_stats_by_faction(db_credentials)


In [32]:
faction_ids

[(500010,), (500004,), (500002,), (500011,), (500003,), (500001,)]

In [33]:
len(faction_dfs[500001])

345